In [2]:
# Enhanced configuration for RAG augmented experiment
MODEL_KEY = 'qwen25'
TOP_K_RULES = 3
MAX_NEW_TOKENS = 128
EXTRA_PROMPT = ''
OUTPUT_DIR = 'outputs'
from utils import ensure_dir
ensure_dir(OUTPUT_DIR)


'outputs'

# Experiment 3: RAG-Augmented Action Decisions

Inject top-k retrieved driving rules (CLIP similarity) before asking for action decision. Compare outputs vs non-RAG.

In [3]:
MODEL_KEY = 'qwen25'
TOP_K_RULES = 3
MAX_NEW_TOKENS = 128
EXTRA_PROMPT = ''

In [4]:
from PIL import Image
from utils import list_frame_pairs, build_single_view_prompt, build_rag_prompt, generate_action, retrieve_rules_for_image, parse_action_json
pairs = list_frame_pairs(folder_name='example_edge_samples')
pairs

[('/home/o0i3z3/thesis/vlm_laboratories/vlm_laboratories/vlm_decision_lab/example_edge_samples/frame01_cam.png',
  '/home/o0i3z3/thesis/vlm_laboratories/vlm_laboratories/vlm_decision_lab/example_edge_samples/frame01_map.png'),
 ('/home/o0i3z3/thesis/vlm_laboratories/vlm_laboratories/vlm_decision_lab/example_edge_samples/frame02_cam.png',
  '/home/o0i3z3/thesis/vlm_laboratories/vlm_laboratories/vlm_decision_lab/example_edge_samples/frame02_map.png'),
 ('/home/o0i3z3/thesis/vlm_laboratories/vlm_laboratories/vlm_decision_lab/example_edge_samples/frame03_cam.png',
  '/home/o0i3z3/thesis/vlm_laboratories/vlm_laboratories/vlm_decision_lab/example_edge_samples/frame03_map.png'),
 ('/home/o0i3z3/thesis/vlm_laboratories/vlm_laboratories/vlm_decision_lab/example_edge_samples/frame04_cam.png',
  '/home/o0i3z3/thesis/vlm_laboratories/vlm_laboratories/vlm_decision_lab/example_edge_samples/frame04_map.png'),
 ('/home/o0i3z3/thesis/vlm_laboratories/vlm_laboratories/vlm_decision_lab/example_edge_sampl

In [5]:
baseline_prompt = build_single_view_prompt(EXTRA_PROMPT)
rag_outputs = []
baseline_outputs = []
parsed_rag = []
parsed_base = []
for cam_path, map_path in pairs:
    cam = Image.open(cam_path).convert('RGB')
    ranked = retrieve_rules_for_image(cam, top_k=TOP_K_RULES)
    rag_prompt = build_rag_prompt(baseline_prompt, ranked, max_rules=TOP_K_RULES)
    base_out = generate_action(MODEL_KEY, [cam], baseline_prompt)
    rag_out = generate_action(MODEL_KEY, [cam], rag_prompt)
    baseline_outputs.append(base_out)
    rag_outputs.append(rag_out)
    parsed_base.append(parse_action_json(base_out) or {})
    parsed_rag.append(parse_action_json(rag_out) or {})
len(rag_outputs), len(baseline_outputs)

/home/o0i3z3/thesis/vlm_laboratories/vlm_laboratories/newvens/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

(6, 6)

In [6]:
parsed_rag

[{'action': 'SLOW',
  'rationale': 'The road is curving, indicating the need to adapt speed to maintain control and visibility.'},
 {'action': 'STOP',
  'rationale': 'The rubber duck obstructs the lane, making it unsafe to proceed forward.'},
 {'action': 'STOP',
  'rationale': "The robot is approaching a curve on the road, and it's important to slow down before curves to ensure safe navigation."},
 {'action': 'SLOW',
  'rationale': 'The road has a curve ahead, requiring reduced speed for safe navigation.'},
 {'action': 'STOP',
  'rationale': 'There are traffic cones blocking the lane, indicating that it is unsafe to proceed forward.'},
 {'action': 'STOP',
  'rationale': 'A yellow duck is on the road, which requires yielding to pedestrians or animals.'}]

In [7]:
# Simple comparison: count action changes introduced by rules
baseline_actions = [p.get('action','?') for p in parsed_base]
rag_actions = [p.get('action','?') for p in parsed_rag]
changes = sum(1 for b,r in zip(baseline_actions, rag_actions) if b != r)
print({'action_changes_due_to_rag': changes})

{'action_changes_due_to_rag': 4}


In [ ]:
# Persist baseline vs RAG comparison summary
from utils import save_text
if 'parsed_rag' in globals():
    import json, os
    base_actions = [p.get('action','?') for p in parsed_base]
    rag_actions = [p.get('action','?') for p in parsed_rag]
    changes = sum(1 for b,r in zip(base_actions, rag_actions) if b != r)
    summary = {
        'baseline_actions': base_actions,
        'rag_actions': rag_actions,
        'action_changes_due_to_rag': changes
    }
    save_text(os.path.join(OUTPUT_DIR,'experiment3_rag_summary.json'), json.dumps(summary, indent=2))
    summary
else:
    print('Run generation cell first.')

: 